In [156]:
!pip install pandas sqlalchemy sqlacodegen

In [94]:
import pickle
import pandas as pd

In [95]:
data=[{"name":"abi"}]

with open("users.pkl",'wb') as f:
    pickle.dump(data,f) #pickle.dump

In [96]:
data_from_pkl=pickle.load(open('users.pkl','rb'))#pickle.loads -> pickle.load
print(data_from_pkl,type(data_from_pkl))

[{'name': 'abi'}] <class 'list'>


In [97]:
df=pd.DataFrame(data)
df.head()


,name
0,abi


In [98]:


df.to_pickle("df_pkl")
df.dtypes

name    object
dtype: object

In [99]:
df2=pd.read_pickle("df_pkl")
df2.head()

,name
0,abi


In [100]:
df2.dtypes



name    object
dtype: object

In [101]:
df.to_csv("my_df.csv",index=False)

###SQLAlchamy

In [102]:
from dataclasses import dataclass

@dataclass
class Movie:
    name:str='Unknown'
    genre:str='Action'
    year:int=None    

In [103]:
movie_obj=Movie(name='interstellar',genre='Sci-Fi')
movie_obj.name

'interstellar'

In [104]:
data=[{
    "name":"intersellar",
    "genre": "Sci-Fi"
},
{
    "name":"The Martian",
    "genre": "Sci-Fi"
},
{
    "name":"Arrival",
    "genre": "Sci-Fi"
}
]

In [106]:
df=pd.DataFrame(data)
df.head()

,name,genre
0,intersellar,Sci-Fi
1,The Martian,Sci-Fi
2,Arrival,Sci-Fi


In [107]:
import sqlalchemy
from sqlalchemy import Column,Integer,String
from sqlalchemy import create_engine,Column,String,Integer
from sqlalchemy.orm import sessionmaker

In [108]:
engine=create_engine("sqlite:///app.db") #mysql,postgres

In [109]:
session =sessionmaker(bind=engine)
my_sess=session()

In [110]:
from sqlalchemy.ext.declarative import declarative_base

Base=declarative_base()

In [111]:

class Movie(Base): #table
    __tablename__="movies"
    
    id=Column(Integer,primary_key=True)
    name=Column(String)#'Unknown'#col
    genre=Column(String)#'Action'#col
    description=Column(String)
    year=Column(Integer,nullable=True) #None #col
    
    
    def __repr__(self):
        return f"<movie name={self.name}>"
    
# save instance=row    

In [112]:
#Add table to database
Base.metadata.create_all(engine)

In [113]:
movie_obj=Movie(name='Interstellar',genre='Sci-Fi')
print(movie_obj.name)

Interstellar


In [114]:
movie_obj.id

In [115]:
my_sess.add(movie_obj) # prepare to save
my_sess.commit() #save

In [116]:
print(movie_obj.id,movie_obj.description)

1 None


In [117]:
movie_obj2=Movie(name='The Martain',genre='Sci-Fi')
print(movie_obj2.name,movie_obj2.id)

The Martain None


In [118]:
my_sess.add(movie_obj2)

In [119]:
movie_obj3=Movie(name='Inception',genre='Sci-Fi')
print(movie_obj3.name,movie_obj3.id)

Inception None


In [120]:
my_sess.add(movie_obj3)

In [121]:
my_sess.commit()

In [122]:
print(movie_obj2.id,movie_obj3.id)

2 3


###CRUD insqlalchemy


In [126]:
Session=sessionmaker(bind=engine)
session=session()

In [131]:
#create 
movie=Movie(name='Guaridans of the Galaxy')
session.add(movie)
session.commit()

In [136]:

# reteive

#get 1 item
movie_a=session.query(Movie).get(5)
print(movie_a.id,movie_a.name)

5 Guaridans of the Galaxy


In [137]:
# List 

qs=session.query(Movie).all()
print(qs)

[<movie name=Interstellar>, <movie name=The Martain>, <movie name=Inception>, <movie name=Guaridans of the Galaxy>, <movie name=Guaridans of the Galaxy>]


In [142]:
# List & Filter 
qs=session.query(Movie).filter_by(name='Guaridans of the Galaxy').all()
qs

[<movie name=Guaridans of the Galaxy>, <movie name=Guaridans of the Galaxy>]

In [143]:
# List & Filter by column value cotaining something
qs=session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
qs

[<movie name=Guaridans of the Galaxy>, <movie name=Guaridans of the Galaxy>]

In [145]:
# List & Filter by column value cotaining something
my_query=input("What are you looking for?") or "Unknow"
qs=session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
print(qs)

What are you looking for?matrix
[<movie name=Guaridans of the Galaxy>, <movie name=Guaridans of the Galaxy>]


In [148]:
#Update

movie_a=session.query(Movie).get(5)
movie_a.description="A dream within a dream"
print(movie_a.id,movie_a.name,movie_a.description)
session.commit()

5 Guaridans of the Galaxy A dream within a dream


In [149]:
movie_a=session.query(Movie).get(5)
print(movie_a.description)

A dream within a dream


In [152]:
qs=session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
for movie_obj in qs:
    movie_obj.name="Guaridans of the Galaxy"
session.commit()    

In [153]:
qs=session.query(Movie).filter(Movie.name.contains("Guaridans")).all()
qs

[<movie name=Guaridans of the Galaxy>, <movie name=Guaridans of the Galaxy>]

In [154]:
#Delete
movie_a=session.query(Movie).get(1)
session.delete(movie_a)
session.commit()

In [155]:
session.flush()

###sql

In [157]:
!sqlacodegen sqlite:///app2.db

# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class Movie(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)


In [159]:
engine=create_engine("sqlite:///app2.db")
Session=sessionmaker(bind=engine)
session=Session()

In [160]:
# coding: utf-8
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()
metadata = Base.metadata


class MovieApp2(Base):
    __tablename__ = 'movies'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    year = Column(Integer)
    
    def __repr__(self):
        return f"<Movie name={self.name}>"

In [161]:
qs=session.query(MovieApp2).all()
print(qs)

[<Movie name=Interstellar>]


In [164]:
for old_obj in qs:

    movie_obj=Movie(name=old_obj.name,genre=old_obj.genre,year=old_obj.year)
    print(movie_obj.name)
    my_sess.add(movie_obj)
my_sess.commit()    

Interstellar


In [165]:
old_engine=create_engine("sqlite:///app2.db")
old_df=pd.read_sql_table("movies",old_engine)
old_df.head()

,id,name,genre,year
0,1,Interstellar,Sci-Fi,None


In [166]:
current_engine=create_engine("sqlite:///app.db")
current_df=pd.read_sql_table("movies",current_engine)
current_df.head()

,id,name,genre,description,year
0,2,The Martain,Sci-Fi,None,None
1,3,Inception,Sci-Fi,None,None
2,4,Guaridans of the Galaxy,None,None,None
3,5,Guaridans of the Galaxy,None,A dream within a dream,None
4,6,Interstellar,Sci-Fi,None,None


In [167]:
final_df=pd.concat([current_df,old_df])
final_df.head(n=20)

,id,name,genre,description,year
0,2,The Martain,Sci-Fi,None,None
1,3,Inception,Sci-Fi,None,None
2,4,Guaridans of the Galaxy,None,None,None
3,5,Guaridans of the Galaxy,None,A dream within a dream,None
4,6,Interstellar,Sci-Fi,None,None
0,1,Interstellar,Sci-Fi,NaN,None


In [168]:
final_df=final_df[['name','genre','description','year']]
final_df.reset_index(inplace=True,drop=True)
final_df.head(n=20)

,name,genre,description,year
0,The Martain,Sci-Fi,None,None
1,Inception,Sci-Fi,None,None
2,Guaridans of the Galaxy,None,None,None
3,Guaridans of the Galaxy,None,A dream within a dream,None
4,Interstellar,Sci-Fi,None,None
5,Interstellar,Sci-Fi,NaN,None


In [172]:
final_df.to_sql(
      "movies",
      current_engine,
    if_exists='replace',
      dtype={
          "name":String,
          "genre":String,
          "year":Integer,
          "description":String,
      }
)

In [173]:
!sqlacodegen sqlite:///app.db

# coding: utf-8
from sqlalchemy import BigInteger, Column, Integer, MetaData, String, Table

metadata = MetaData()


t_movies = Table(
    'movies', metadata,
    Column('index', BigInteger, index=True),
    Column('name', String),
    Column('genre', String),
    Column('description', String),
    Column('year', Integer)
)


t_movies_2 = Table(
    'movies_2', metadata,
    Column('index', BigInteger, index=True),
    Column('name', String),
    Column('genre', String),
    Column('description', String),
    Column('year', Integer)
)


In [176]:
import sqlite3


conx=sqlite3.connect('app.db')
conx2=create_engine("sqlite:///app.db")

df=pd.read_sql_query("SELECT * FROM movies",conx)
df.head()

,index,name,genre,description,year
0,0,The Martain,Sci-Fi,None,None
1,1,Inception,Sci-Fi,None,None
2,2,Guaridans of the Galaxy,None,None,None
3,3,Guaridans of the Galaxy,None,A dream within a dream,None
4,4,Interstellar,Sci-Fi,None,None


In [177]:
#cfe.sh/youtube